# Coupling NGSolve and NGSpice

In [ ]:
from netgen.occ import *
outer = MoveTo(0,0).RectangleC(0.1,0.1).Face()
el1 = MoveTo(0,0.002).RectangleC(0.03,0.001).Face()
el2 = MoveTo(0,-0.002).RectangleC(0.03,0.001).Face()
el1.edges.name="el1"
el2.edges.name="el2"

dom = outer-el1-el2
geom = OCCGeometry(dom, dim=2)

In [ ]:
from ngsolve import *
from ngsolve.webgui import Draw
mesh = Mesh(geom.GenerateMesh(maxh=0.005))
# Draw (mesh);

In [ ]:
fes = H1(mesh, order=3, dirichlet="el1|el2")
u,v = fes.TnT()
gfu = GridFunction(fes)
gfu.Set (mesh.BoundaryCF({"el1":1, "el2":0}), BND)

mata = BilinearForm(10*8.854e-12*grad(u)*grad(v)*dx).Assemble().mat
inv = mata.Inverse(inverse="sparsecholesky", freedofs=fes.FreeDofs())
gfu.vec.data -= inv@mata * gfu.vec

capacity = InnerProduct(mata*gfu.vec, gfu.vec)
print ("capacity=", capacity)
Draw (gfu);

In [ ]:
try:
    from PySpice.Spice.Netlist import Circuit
    from PySpice.Unit import *
except:
    pass

In [ ]:
try:
    circuit = Circuit('test1')

    source = circuit.SinusoidalVoltageSource(1, 'input', circuit.gnd, amplitude=1@u_V, frequency=1@u_kHz)
    # source = circuit.PulseVoltageSource(1, 'input', circuit.gnd, 1, 2, 0, 0, 0, 0.001, 0.01)
    
    circuit.R(1, 'input', 'output', 100@u_kOhm)
    # circuit.C(1, 'output', 0, 1@u_nF)
    circuit.C(1, 'output', circuit.gnd, capacity)

    print (circuit)
except:
    pass

In [ ]:
try:
    simulator = circuit.simulator()
    analysis = simulator.transient(step_time=source.period/50, end_time=source.period*4)
    # analysis = simulator.transient(step_time=1e-4, end_time=1e-2)
    
    import matplotlib.pyplot as plt
    plt.plot (analysis.input)
    plt.plot (analysis.output)
except:
    pass